In [50]:
import pandas as pd
import numpy as np 
import re
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
import string
from emoji import UNICODE_EMOJI
from spellchecker import SpellChecker
from textblob import TextBlob
from textblob import Word
spell = SpellChecker()
nltk.download('brown')
from string import punctuation
from collections import Counter
#from keras.preprocessing.text import text_to_word_sequence, one_hot, hashing_trick, Tokenizer
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\iamch\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


# Data Prep

We start by loading the data, splitting the comments to be individual data points. 


In [3]:
mbti_data_original = pd.read_csv('../Data/mbti_data.csv')
yelp_data = pd.read_csv('../Data/yelp_comments_reduced.csv')
print(mbti_data_original.shape)
print(yelp_data.shape)

(8675, 2)
(3303, 3)


In [4]:
mbti_split = mbti_data_original



In [5]:
p_types = mbti_split['type'].unique()


In [6]:
stop = stopwords.words('english')

def avg_word(sentence):
  words = sentence.split()
  if len(words) > 0:
      return round(sum(len(word) for word in words)/len(words),2)
  return 0

def get_misspelled_details(split_word, word_count):
    misspelled = spell.unknown(split_word)
    if len(misspelled) > 0:
        average_length_misspelled = round(sum(len(word) for word in misspelled)/len(misspelled),2)
    else:
        average_length_misspelled = -1
    return(len(misspelled) / word_count), average_length_misspelled
    
def lemmatise_n_spell_check(word_list):
   # return [Word(spell.correction(word)).lemmatize() for word in word_list]
    return [Word(word).lemmatize() for word in word_list]

def remove_punctuation(entry):
    tab = str.maketrans(dict.fromkeys(string.punctuation))
    return entry.translate(tab)  

def count_emoji(word_list):
    emoji_count = 0
    for emoji in UNICODE_EMOJI:
        emoji_count += word_list.count(emoji)
    return emoji_count

def get_word_probabilities(split_word):
    word_probs = np.array([spell.word_probability(x) for x in split_word])
    word_probabilities = word_probs[word_probs > 0.00001]
    max_word_prob = max(word_probabilities)
    average_word_prob = np.mean(word_probabilities)
    lowest_word_prob = min(word_probabilities)
    std_word_prob = np.std(word_probabilities)
    return max_word_prob, average_word_prob, lowest_word_prob, std_word_prob
    
def get_sentiments(doc):
    
    text_blob = TextBlob(doc)
    polarity = []
    subjectivity = [] 
    for sentence in text_blob.sentences:
        polarity.append(sentence.sentiment.polarity)
        subjectivity.append(sentence.sentiment.subjectivity)

    
    return max(polarity), np.mean(polarity), min(polarity), max(subjectivity), np.mean(subjectivity), min(subjectivity)

# turn a doc into clean tokens
def clean_comment(doc):
    for types in p_types:
        doc = re.sub(types,  'type',    doc)  
        doc = re.sub(types.lower(), 'type', doc)
    
    count = lambda l1, l2: len(list(filter(lambda c: c in l2, l1))) ##counter

    doc = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', doc, flags=re.MULTILINE)
    doc = re.sub('[|]', '', doc)
    max_polarity, average_polarity, min_polarity,  max_subjectivity, avg_subjectivity, min_subjectivity = get_sentiments(doc)
   
    punctuation_count = count(doc, string.punctuation)
    split_word = doc.split()
    word_count = len(split_word)
    char_count = len(doc)
    av_word = avg_word(doc)
    
    max_word_prob, average_word_prob, lowest_word_prob, std_word_prob = get_word_probabilities(split_word)

    
    emoji_count = count_emoji(doc)
    
    numerics = len([x for x in split_word if x.isdigit()])  
    stop_words = len([x for x in split_word if x in stop])
    upper = len([x for x in split_word if x.isupper() & len(x) > 1])
    

    doc = remove_punctuation(doc)
    tokens = re.sub("[^\w]", " ",  doc).split()
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [w for w in tokens if not w in stop] 

    percentage_misspelled, avg_misspelled = get_misspelled_details(split_word, word_count)
    tokens = lemmatise_n_spell_check(tokens)

    return {'tokens' : tokens, 
            'max_polarity' : round(max_polarity,3),
            'average_polarity' : round(average_polarity,3),
            'min_polarity' : round(min_polarity,2),
            'max_subjectivity' : round(max_subjectivity,3),
            'average_subjectivity' : round(avg_subjectivity,3),
            'min_subjectivity' : round(min_subjectivity,3), 
            'misspelled%' : round(percentage_misspelled,3),
            'average_misspelled_word_length' : round(avg_misspelled,3),
            'emoji_%' : round(emoji_count / char_count, 3),
            'puncutation_%' : round(punctuation_count / char_count,3),
            'average word length' : av_word,
            'highest_word_probability' : round(max_word_prob,3),
            'average_word_probability' : round(average_word_prob,3),
            'lowest_word_probability' : round(lowest_word_prob,5),
            'std_word_probability' : round(std_word_prob,3),
            'number %' : round(numerics / word_count,3),
            'stop word %' : round(stop_words / word_count, 3),
            'upper word %' : round(upper / word_count ,3)}

In [7]:
#clean_comment(' I am so sad. This is truely terible and yet. I still have 1 or maybe even 2 emojis (bt I cant spell so well) 😁') 
clean_comment(mbti_split.iloc[4015]['posts'])
clean_comment(mbti_split.iloc[4016]['posts'])
clean_comment(mbti_split.iloc[4014]['posts'])

{'tokens': ['day',
  'look',
  'back',
  'laugh',
  'pretty',
  'insane',
  'strong',
  'depression',
  'fueled',
  'hatred',
  'anger',
  'bei',
  'discovered',
  'im',
  'really',
  'much',
  'type',
  'thought',
  'think',
  'go',
  'back',
  'lurking',
  'every',
  'best',
  'song',
  'eversometimes',
  'think',
  'im',
  'type',
  'im',
  'like',
  'others',
  'dont',
  'think',
  'fit',
  'anywhereusually',
  'laying',
  'back',
  'nice',
  'get',
  'yes',
  'handful',
  'time',
  'people',
  'didnt',
  'like',
  'methey',
  'would',
  'talk',
  'smack',
  'course',
  'holdmy',
  'usual',
  'self',
  'back',
  'day',
  'would',
  'show',
  'hysterical',
  'side',
  'group',
  'friend',
  'would',
  'say',
  'lol',
  'imagine',
  'highi',
  'bit',
  'extroverted',
  'yes',
  'whileyes',
  'yes',
  'yes',
  'put',
  'away',
  'swordhmmso',
  'read',
  'girl',
  'said',
  'place',
  'pretty',
  'much',
  'nice',
  'dont',
  'clingy',
  'insecure',
  'constitutes',
  'clingy',
  'nci

We next split the comments into word vectors, find the total number of unique words then use the md5 hash function to create an integer hash of each of the words 

In [8]:
cleaned_mbti = mbti_split['posts'].apply(clean_comment)
cleaned_yelp = yelp_data['text'].apply(clean_comment)



In [20]:
def data_frame_the_cleaned_data(data, cleaned):
    data['posts'] = [item['tokens'] for item in cleaned]
    data['posts'].replace('[]', np.nan, inplace=True)
    data['joined_comment'] = data['posts'].apply(lambda x: " ".join(x))
    data.dropna(subset=['posts'], inplace=True)

    data['max_polarity'] = [item['max_polarity'] for item in cleaned]
    data['average_polarity'] = [item['average_polarity'] for item in cleaned]
    data['min_polarity'] = [item['min_polarity'] for item in cleaned]

    data['max_subjectivity'] = [item['max_subjectivity'] for item in cleaned]
    data['average_subjectivity'] = [item['average_subjectivity'] for item in cleaned]
    data['min_subjectivity'] = [item['min_subjectivity'] for item in cleaned]

    data['misspelled%'] = [item['misspelled%'] for item in cleaned]
    data['average_misspelled_word_length'] = [item['average_misspelled_word_length'] for item in cleaned]
    data['emoji_%'] = [item['emoji_%'] for item in cleaned]
    data['puncutation_%'] = [item['puncutation_%'] for item in cleaned]

    data['average word length'] = [item['average word length'] for item in cleaned]
    data['highest_word_probability'] = [item['highest_word_probability'] for item in cleaned]
    data['average_word_probability'] = [item['average_word_probability'] for item in cleaned]
    data['std_word_probability'] = [item['std_word_probability'] for item in cleaned]

    data['number %'] = [item['number %'] for item in cleaned]
    data['stop word %'] = [item['stop word %'] for item in cleaned]
    data[ 'upper word %'] = [item[ 'upper word %'] for item in cleaned]
    return data

In [21]:
mbti_split = data_frame_the_cleaned_data(mbti_split, cleaned_mbti) 
yelp_data = data_frame_the_cleaned_data(yelp_data, cleaned_yelp) 


In [22]:

print(mbti_split.head())

   type                                              posts  \
0  INFJ  [type, moment, sportscenter, top, ten, play, p...   
1  ENTP  [im, finding, lack, post, alarmingsex, boring,...   
2  INTP  [good, one, course, say, know, thats, blessing...   
3  INTJ  [dear, type, enjoyed, conversation, day, esote...   
4  ENTJ  [youre, firedthats, another, silly, misconcept...   

                                      joined_comment  max_polarity  \
0  type moment sportscenter top ten play prankswh...          0.55   
1  im finding lack post alarmingsex boring positi...          1.00   
2  good one course say know thats blessing cursed...          1.00   
3  dear type enjoyed conversation day esoteric ga...          0.80   
4  youre firedthats another silly misconception a...          1.00   

   average_polarity  min_polarity  max_subjectivity  average_subjectivity  \
0             0.089          -0.5               1.0                 0.400   
1             0.123          -1.0               1.0 

In [23]:
b_o_w_vec = CountVectorizer(max_features=5000,  lowercase=True, ngram_range=(1,1), analyzer = "word")
b_o_w_mbti = b_o_w_vec.fit_transform(mbti_split['joined_comment'])
b_o_w_yelp = b_o_w_vec.transform(yelp_data['joined_comment'])

In [24]:
print(b_o_w_vec.vocabulary_)

{'type': 4634, 'moment': 2848, 'top': 4540, 'ten': 4397, 'play': 3288, 'experience': 1545, 'life': 2589, 'repeat': 3648, 'perc': 3207, 'youthe': 4989, 'last': 2521, 'thing': 4466, 'friend': 1779, 'posted': 3345, 'facebook': 1583, 'suicide': 4275, 'next': 2960, 'day': 1094, 'rest': 3687, 'peace': 3193, 'sorry': 4083, 'hear': 2015, 'natural': 2922, 'relationship': 3618, 'perfection': 3217, 'time': 4509, 'every': 1498, 'existence': 1535, 'try': 4615, 'figure': 1657, 'hard': 1987, 'growth': 1934, 'welcome': 4845, 'stuff': 4240, 'game': 1814, 'set': 3881, 'least': 2559, 'minute': 2821, 'moving': 2878, 'leg': 2566, 'dont': 1284, 'mean': 2749, 'sitting': 3983, 'desk': 1170, 'chair': 688, 'weed': 4837, 'maybe': 2746, 'come': 834, 'three': 4486, 'item': 2391, 'youve': 4991, 'determined': 1184, 'want': 4805, 'would': 4939, 'likely': 2598, 'use': 4714, 'given': 1863, 'cognitive': 814, 'function': 1800, 'whatnot': 4856, 'left': 2565, 'sims': 3967, 'indeed': 2234, 'video': 4763, 'good': 1879, 'one'

In [25]:
b_o_w_processed_mbti = pd.DataFrame(b_o_w_mbti.toarray(), columns = b_o_w_vec.get_feature_names())
b_o_w_processed_yelp = pd.DataFrame(b_o_w_yelp.toarray(), columns = b_o_w_vec.get_feature_names())


In [26]:
def split_type(x):
    return [x[0], x[1], x[2], x[3]]


In [27]:
t = mbti_split['type'].apply(lambda x : split_type(x))
mbti_split['t1'] = [item[0] for item in t]
mbti_split['t2'] = [item[1] for item in t]
mbti_split['t3'] = [item[2] for item in t]
mbti_split['t4'] = [item[3] for item in t]

mbti_split.head(1000)

,type,posts,joined_comment,max_polarity,average_polarity,min_polarity,max_subjectivity,average_subjectivity,min_subjectivity,misspelled%,...,highest_word_probability,average_word_probability,std_word_probability,number %,stop word %,upper word %,t1,t2,t3,t4
0,INFJ,"[type, moment, sportscenter, top, ten, play, p...",type moment sportscenter top ten play prankswh...,0.550,0.089,-0.50,1.000,0.400,0.0,0.217,...,0.064,0.009,0.016,0.002,0.403,0.0,I,N,F,J
1,ENTP,"[im, finding, lack, post, alarmingsex, boring,...",im finding lack post alarmingsex boring positi...,1.000,0.123,-1.00,1.000,0.394,0.0,0.193,...,0.064,0.009,0.015,0.001,0.398,0.0,E,N,T,P
2,INTP,"[good, one, course, say, know, thats, blessing...",good one course say know thats blessing cursed...,1.000,0.122,-1.00,1.000,0.444,0.0,0.222,...,0.064,0.008,0.014,0.006,0.371,0.0,I,N,T,P
3,INTJ,"[dear, type, enjoyed, conversation, day, esote...",dear type enjoyed conversation day esoteric ga...,0.800,0.077,-0.50,1.000,0.359,0.0,0.202,...,0.064,0.009,0.015,0.005,0.405,0.0,I,N,T,J
4,ENTJ,"[youre, firedthats, another, silly, misconcept...",youre firedthats another silly misconception a...,1.000,0.045,-0.80,1.000,0.379,0.0,0.201,...,0.064,0.008,0.014,0.001,0.426,0.0,E,N,T,J
5,INTJ,"[science, perfect, scientist, claim, scientifi...",science perfect scientist claim scientific inf...,1.000,0.041,-1.00,1.000,0.352,0.0,0.206,...,0.064,0.008,0.013,0.001,0.424,0.0,I,N,T,J
6,INFJ,"[cant, draw, nail, haha, done, professional, n...",cant draw nail haha done professional nail yes...,0.800,0.076,-0.50,1.000,0.376,0.0,0.188,...,0.064,0.007,0.013,0.007,0.406,0.0,I,N,F,J
7,INTJ,"[tend, build, collection, thing, desktop, use,...",tend build collection thing desktop use freque...,0.700,0.060,-0.31,0.744,0.272,0.0,0.147,...,0.064,0.008,0.014,0.002,0.463,0.0,I,N,T,J
8,INFJ,"[im, sure, thats, good, question, distinction,...",im sure thats good question distinction two de...,0.875,0.179,-0.39,1.000,0.495,0.0,0.224,...,0.064,0.009,0.015,0.000,0.413,0.0,I,N,F,J
9,INTP,"[position, actually, let, go, person, due, var...",position actually let go person due various re...,1.000,0.149,-0.50,1.000,0.403,0.0,0.185,...,0.064,0.009,0.015,0.002,0.442,0.0,I,N,T,P


In [30]:
mbti_processed = pd.concat([mbti_split, b_o_w_processed_mbti ], axis=1, sort=False)
yelp_processed = pd.concat([yelp_data, b_o_w_processed_yelp ], axis=1, sort=False)

In [31]:
mbti_processed.to_csv('../Data/mbti_pre-processed.csv')
yelp_processed.to_csv('../Data/yelp_pre-processed.csv')

In [36]:
mbti_E = mbti_processed[mbti_processed['t1'] == 'E']
mbti_I = mbti_processed[mbti_processed['t1'] == 'I']
mbti_N = mbti_processed[mbti_processed['t2'] == 'N']
mbti_S = mbti_processed[mbti_processed['t2'] == 'S']
mbti_F = mbti_processed[mbti_processed['t3'] == 'F']
mbti_T = mbti_processed[mbti_processed['t3'] == 'T']
mbti_P = mbti_processed[mbti_processed['t4'] == 'P']
mbti_J = mbti_processed[mbti_processed['t4'] == 'J']

e_i_count = min(len(mbti_E), len(mbti_I))
n_s_count = min(len(mbti_N), len(mbti_S))
f_t_count = min(len(mbti_F), len(mbti_T))
p_j_count = min(len(mbti_P), len(mbti_J))

print(e_i_count, n_s_count, f_t_count, p_j_count)

1999 1197 3981 3434


In [58]:
mbti_EI = pd.concat([mbti_E.sample(e_i_count), mbti_I.sample(e_i_count)], ignore_index=True)
mbti_EI_target = mbti_EI['t1']
mbti_EI.drop(['type', 'posts', 'joined_comment', 't1', 't2', 't3', 't4'], axis = 1)

mbti_NS = pd.concat([mbti_N.sample(n_s_count), mbti_S.sample(n_s_count)],ignore_index=True)
mbti_NS_target = mbti_NS['t2']
mbti_NS.drop(['type', 'posts', 'joined_comment', 't1', 't2', 't3', 't4'], axis = 1)


mbti_FT = pd.concat([mbti_F.sample(f_t_count), mbti_T.sample(f_t_count)],ignore_index=True)
mbti_FT_target = mbti_FT['t3']
mbti_FT.drop(['type', 'posts', 'joined_comment', 't2', 't1', 't3', 't4'], axis = 1)

mbti_PJ = pd.concat([mbti_P.sample(p_j_count), mbti_J.sample(p_j_count)],ignore_index=True)
mbti_PJ_target = mbti_PJ['t4']
mbti_PJ.drop(['type', 'posts', 'joined_comment', 't2', 't3', 't1', 't4'], axis = 1)

,max_polarity,average_polarity,min_polarity,max_subjectivity,average_subjectivity,min_subjectivity,misspelled%,average_misspelled_word_length,emoji_%,puncutation_%,...,youtube,youve,youyou,yr,yup,zen,zero,zodiac,zombie,zone
0,0.700,0.120,-0.50,1.000,0.421,0.0,0.312,6.55,0.0,0.042,...,1,0,0,0,0,0,0,0,0,0
1,0.700,0.084,-0.60,1.000,0.311,0.0,0.223,6.77,0.0,0.055,...,0,1,0,0,0,0,0,0,0,0
2,1.000,0.213,-0.25,1.000,0.440,0.0,0.224,6.33,0.0,0.056,...,0,0,0,0,0,0,0,0,2,0
3,0.800,0.075,-0.42,1.000,0.338,0.0,0.181,7.13,0.0,0.053,...,0,1,0,0,0,0,0,0,0,0
4,0.646,0.118,-0.31,0.889,0.328,0.0,0.179,6.72,0.0,0.052,...,0,0,0,0,0,0,0,0,0,0
5,0.800,0.067,-0.70,1.000,0.385,0.0,0.201,6.60,0.0,0.045,...,0,1,0,0,0,0,0,0,0,0
6,0.700,0.092,-0.46,1.000,0.440,0.0,0.170,7.06,0.0,0.041,...,0,1,0,0,0,0,0,0,0,0
7,0.850,0.106,-0.80,1.000,0.354,0.0,0.186,6.86,0.0,0.046,...,0,0,0,0,0,0,0,0,0,0
8,0.900,0.141,-0.50,1.000,0.348,0.0,0.240,6.96,0.0,0.054,...,0,1,0,0,0,0,0,0,0,1
9,0.910,0.076,-0.88,1.000,0.382,0.0,0.173,7.03,0.0,0.042,...,0,1,0,0,0,0,0,0,0,0


In [51]:
mbti_EI.head()

,type,posts,joined_comment,max_polarity,average_polarity,min_polarity,max_subjectivity,average_subjectivity,min_subjectivity,misspelled%,...,youtube,youve,youyou,yr,yup,zen,zero,zodiac,zombie,zone
0,ENFP,"[conversation, aways, deep, talking, world, ne...",conversation aways deep talking world news c p...,0.800,0.105,-0.80,1.0,0.402,0.0,0.210,...,0,1,0,0,0,0,0,0,0,0
1,ENTP,"[oh, come, idontthinkso, cute, little, mouse, ...",oh come idontthinkso cute little mouse making ...,0.600,0.073,-0.58,1.0,0.363,0.0,0.215,...,0,5,0,0,0,0,0,0,0,0
2,ENFP,"[would, put, contra, type, personally, said, r...",would put contra type personally said regardin...,0.600,0.041,-0.60,1.0,0.405,0.0,0.236,...,0,0,0,0,0,0,0,0,0,0
3,ENTJ,"[thought, fucking, sx, hostility, ring, freaki...",thought fucking sx hostility ring freaking ima...,0.700,0.078,-0.60,1.0,0.368,0.0,0.208,...,0,1,0,0,0,0,0,0,0,0
4,ESFP,"[good, job, william, amyes, selfinterest, self...",good job william amyes selfinterest selflessne...,0.875,0.140,-1.00,1.0,0.418,0.0,0.187,...,0,0,0,0,0,0,0,0,0,0


In [59]:
mbti_EI_train, mbti_EI_test, mbti_EI_train_target, mbti_EI_test_target = train_test_split(mbti_EI, mbti_EI_target,test_size = 0.2)
mbti_NS_train, mbti_NS_test, mbti_NS_train_target, mbti_NS_test_target = train_test_split(mbti_NS, mbti_NS_target, test_size = 0.2)
mbti_FT_train, mbti_FT_test, mbti_FT_train_target, mbti_FT_test_target = train_test_split(mbti_FT,mbti_FT_target, test_size = 0.2)
mbti_PJ_train, mbti_PJ_test, mbti_PJ_train_target, mbti_PJ_test_target = train_test_split(mbti_PJ, mbti_PJ_target, test_size = 0.2)

In [61]:
mbti_EI_train.to_csv('../Data/traintest/mbti_EI_train')
mbti_EI_test.to_csv('../Data/traintest/mbti_EI_test')
mbti_EI_train_target.to_csv('../Data/traintest/mbti_EI_train_target')
mbti_EI_test_target.to_csv('../Data/traintest/mbti_EI_test_target')

mbti_NS_train.to_csv('../Data/traintest/mbti_NS_train')
mbti_NS_test.to_csv('../Data/traintest/mbti_NS_test')
mbti_NS_train_target.to_csv('../Data/traintest/mbti_NS_train_target')
mbti_NS_test_target.to_csv('../Data/traintest/mbti_NS_test_target')

mbti_FT_train.to_csv('../Data/traintest/mbti_FT_train')
mbti_FT_test.to_csv('../Data/traintest/mbti_FT_test')
mbti_FT_train_target.to_csv('../Data/traintest/mbti_FT_train_target')
mbti_FT_test_target.to_csv('../Data/traintest/mbti_FT_test_target')

mbti_PJ_train.to_csv('../Data/traintest/mbti_PJ_train')
mbti_PJ_test.to_csv('../Data/traintest/mbti_PJ_test')
mbti_PJ_train_target.to_csv('../Data/traintest/mbti_PJ_train_target')
mbti_PJ_test_target.to_csv('../Data/traintest/mbti_PJ_test_target')

In [62]:
mbti_PJ_train.shape

(5494, 5024)